In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from shutil import copyfile
import pathlib
from PIL import Image
import datetime
from tqdm import tqdm

In [2]:
def convert_dataset(path_from, path_to):
    for img in tqdm(os.listdir(path_from)):
        try:
            if os.path.splitext(img)[-1] == '.png':
                im = Image.open(os.path.join(path_from, img))
                rgb_im = im.convert('RGB')
                rgb_im.save(os.path.join(path_to, '{}.jpg'.format(os.path.splitext(img)[0])), quality=100)
        except:
            pass

In [3]:
convert_dataset('./images_png/', './images/')

100%|██████████| 15031/15031 [03:56<00:00, 63.61it/s]


In [17]:
datapath = "./data/"
all_ships = pd.read_csv(os.path.join(datapath, "all_ships.csv"))
all_ships.drop(columns=['COURSE', 'DESTINATION', 'DWT', 'ELAPSED', 'FLAG', 'GT_SHIPTYPE',
                        'HEADING', 'LAT', 'LEGEND', 'LENGTH', 'LON', 'L_FORE', 'ROT',
                        'SHIPNAME', 'SPEED', 'STATUS_NAME','WIDTH', 'W_LEFT'], inplace=True)


In [18]:
all_ships['DIRNAME'] = all_ships.apply(lambda x: "{}_{}".format(x["SHIPTYPE"], x["TYPE_NAME"]), axis=1)
all_ships['OLD_IMG_PATH'] = all_ships.apply(lambda x: "./images/{}.jpg".format(x['SHIP_ID']), axis=1)

In [19]:
train_valid, test = train_test_split(all_ships, test_size=0.2)
train, valid = train_test_split(train_valid, test_size=0.2)

In [20]:
dfs = [train, valid, test]
paths = ['./shipImages/train', './shipImages/valid', './shipImages/test']
for (df, path) in tqdm(zip(dfs, paths)):
    df['NEW_IMG_PATH'] = df.apply(lambda x: "{}/{}/".format(path, x['DIRNAME']), axis=1)
    df['NEW_IMG_NAME'] = df.apply(lambda x: "{}_{}.jpg".format(x['TYPE_NAME'],x['SHIP_ID']), axis=1)


0it [00:00, ?it/s]/home/saghar/.virtualenvs/02ML/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/saghar/.virtualenvs/02ML/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
3it [00:02,  1.06it/s]


In [21]:
for (df, path) in zip(dfs, paths):
    for (src, dstpath, dstname) in tqdm(zip(df['OLD_IMG_PATH'], df['NEW_IMG_PATH'], df['NEW_IMG_NAME'])):
        dst = os.path.join(dstpath, dstname)
        pathlib.Path(dstpath).mkdir(parents=True, exist_ok=True)
        if os.path.exists(src):
            copyfile(src, dst)
    

33737it [00:09, 3551.01it/s] 
8435it [00:04, 1948.41it/s]
10543it [00:04, 2363.40it/s]
